TP1

Laboratorio de Datos

1er Cuatrimestre - 2024

Iván Exequiel Pintos - Joaquín Rovner - Juan José García Vizioli

- Importamos todas las librerías que venimos usando:

In [ ]:
import seaborn as sns
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

from scipy import stats
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from formulaic import model_matrix

- Leemos el dataset:

In [ ]:
#@title Usando Colab

from google.colab import drive
drive.mount('/content/drive')

sube_2023_file_path = '/content/drive/My Drive/LDD-TP1/sube-2023.csv'

Mounted at /content/drive


In [ ]:
#@title Usando Jupyter

sube_2023_file_path = '../LDD-TP1/sube-2023.csv'

In [ ]:
SUBE_2023 = pd.read_csv(sube_2023_file_path)

#Procesamiento de Datos

##1)

###a)

Texto 1)a)

###b)

####i)

Texto 1)b)i)

####ii)


Texto 1)b)ii)

####iii)

Texto 1)b)iii)

##2)

Texto 2)

##3)

###a)

Texto 3)a)

###b)

Texto 3)b)

###c)

Texto 3)c)

#Análisis Exploratorio

##4)

Texto 4)

#Modelado

##5)

###a)

Texto 5)a)

###b)

Texto 5)b)

###c)

Texto 5)c)

###d)

Texto 5)d)

###e)

Texto 5)e)

###f)

Texto 5)f)

###g)

Texto 5)g)